In [54]:
from sklearn import set_config
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RandomizedSearchCV,
    train_test_split,
)
from sklearn.pipeline import Pipeline
from scipy.stats import loguniform

from skfolio import RatioMeasure, RiskMeasure
from skfolio.datasets import load_factors_dataset, load_sp500_dataset
from skfolio.model_selection import (
    CombinatorialPurgedCV,
    WalkForward,
    cross_val_predict,
)
from skfolio.moments import (
    DenoiseCovariance,
    DetoneCovariance,
    EWMu,
    GerberCovariance,
    ShrunkMu,
)
from skfolio.optimization import (
    MeanRisk,
    NestedClustersOptimization,
    ObjectiveFunction,
    RiskBudgeting,
)
from skfolio.pre_selection import SelectKExtremes # type: ignore
from skfolio.preprocessing import prices_to_returns
from skfolio.prior import BlackLitterman, EmpiricalPrior, FactorModel
from skfolio.uncertainty_set import BootstrapMuUncertaintySet

# Loading some dataset

In [55]:
prices = load_sp500_dataset()

In [56]:
prices

,AAPL,AMD,BAC,BBY,CVX,GE,HD,JNJ,JPM,KO,LLY,MRK,MSFT,PEP,PFE,PG,RRC,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,
1990-01-02,0.264,4.125,4.599,0.144,4.991,14.391,1.117,3.438,3.394,2.235,6.658,4.215,0.384,4.738,1.021,3.860,3.322,0.310,3.653,4.068
1990-01-03,0.266,4.000,4.636,0.161,4.910,14.364,1.121,3.452,3.508,2.203,6.658,4.282,0.386,4.692,1.024,3.853,3.322,0.304,3.653,4.027
1990-01-04,0.267,3.938,4.537,0.159,4.847,14.283,1.128,3.459,3.522,2.192,6.621,4.215,0.397,4.646,1.041,3.777,3.322,0.301,3.634,3.987
1990-01-05,0.268,3.812,4.438,0.159,4.775,14.148,1.113,3.423,3.536,2.174,6.549,4.128,0.387,4.581,1.032,3.709,3.322,0.288,3.595,3.966
1990-01-08,0.269,3.812,4.463,0.147,4.820,14.229,1.102,3.481,3.536,2.220,6.549,4.181,0.393,4.664,1.023,3.777,3.322,0.282,3.644,4.027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-21,135.057,67.680,32.212,78.729,171.490,64.670,314.798,175.090,129.282,62.836,365.872,109.611,243.287,180.017,50.084,149.015,26.574,523.519,144.040,106.312
2022-12-22,131.846,63.860,31.927,78.563,168.918,63.727,311.604,174.450,127.814,62.383,363.187,109.739,237.077,178.627,50.065,149.359,25.232,523.072,142.354,104.168
2022-12-23,131.477,64.520,32.005,79.432,174.140,63.742,314.177,174.893,128.421,62.855,365.762,110.350,237.614,179.781,50.249,149.781,26.226,527.260,142.641,106.922


train/test split

In [57]:
X = prices_to_returns(prices)
X_train, X_test = train_test_split(X, test_size=0.33, shuffle=False)

Specifying model.

This does some form of convex optimizartion

In [58]:
model = MeanRisk(
     objective_function=ObjectiveFunction.MAXIMIZE_UTILITY,
     risk_measure=RiskMeasure.CVAR,
     risk_aversion= 0.5,
     budget=1.0,
     max_short=0.2,
     risk_free_rate=0.04
)

In [59]:
model.fit(X_train)

print(model.weights_)

[2.52034400e-02 5.26348601e-10 1.67998926e-10 1.83981695e-02
 1.30850276e-01 5.45501224e-10 2.07125647e-09 2.11231000e-01
 2.44488746e-10 6.35825295e-02 1.80213285e-02 1.22665375e-09
 4.73097124e-09 1.28081376e-01 1.28488703e-09 1.46384582e-01
 1.76368751e-02 2.06557039e-02 1.13900775e-01 1.06053933e-01]


In [60]:
portfolio = model.predict(X_test)

print(portfolio.annualized_sharpe_ratio)
print(portfolio.summary())

0.6249558811602138
Mean                                             0.053%
Annualized Mean                                  13.37%
Variance                                        0.0089%
Annualized Variance                               2.25%
Semi-Variance                                   0.0046%
Annualized Semi-Variance                          1.15%
Standard Deviation                                0.94%
Annualized Standard Deviation                    14.99%
Semi-Deviation                                    0.68%
Annualized Semi-Deviation                        10.73%
Mean Absolute Deviation                           0.61%
CVaR at 95%                                       2.17%
EVaR at 95%                                       4.60%
Worst Realization                                 8.57%
CDaR at 95%                                      14.62%
MAX Drawdown                                     33.70%
Average Drawdown                                  3.08%
EDaR at 95%                  

In [61]:
portfolio.plot_returns()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'variable=returns<br>index=%{x}<br>value=%{y}<extra></extra>',
              'legendgroup': 'returns',
              'line': {'color': '#636efa', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': 'returns',
              'showlegend': True,
              'type': 'scattergl',
              'x': array([datetime.datetime(2012, 2, 6, 0, 0),
                          datetime.datetime(2012, 2, 7, 0, 0),
                          datetime.datetime(2012, 2, 8, 0, 0), ...,
                          datetime.datetime(2022, 12, 23, 0, 0),
                          datetime.datetime(2022, 12, 27, 0, 0),
                          datetime.datetime(2022, 12, 28, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([ 0.00356727,  0.00362919, -0.00088711, ...,  0.0104581 ,  0.00506986,
                          -0.01256507]),
              'yaxis': 'y'}],
    'layout': {'legend': {'title': {'text': 'variable'}, 'tracegroupgap': 0},
               'margin': {'t': 60},
               'showlegend': False,
               'template': '...',
               'title': {'text': 'Returns'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'Observations'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Returns'}}}
})